<a href="https://colab.research.google.com/github/link-kut/deeplink_public/blob/master/2.ReinforcementLearning/CartPole/a3c_cartpole_tf20_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### https://medium.com/@kaleajit27/reinforcement-learning-on-google-colab-9cb2e1ef51e
### https://colab.research.google.com/drive/1flu31ulJlgiRL1dnN2ir8wGh9p7Zij2t#scrollTo=8nj5sjsk15IT

In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [2]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (41.0.1)


In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [4]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")
    

def wrap_env(env):
    env = Monitor(env, './video', force=True)
    return env

In [0]:
env = wrap_env(gym.make("CartPole-v0"))
#env = wrap_env(gym.make("MsPacman-v0"))

In [12]:
observation = env.reset()

while True:
    env.render()
    
    #your agent goes here
    action = env.action_space.sample() 
         
    observation, reward, done, info = env.step(action)
    
    if done: 
        break;
            
env.close()
show_video()

In [8]:
!pip install tensorflow-gpu==2.0.0-alpha0
!pip install gputil
!pip install psutil
!pip install humanize

     |████████████████████████████████| 332.1MB 56kB/s 
     |████████████████████████████████| 419kB 38.7MB/s 
     |████████████████████████████████| 3.0MB 45.7MB/s 
     |████████████████████████████████| 61kB 26.3MB/s 


  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [0]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as kl

In [0]:
a = tf.random.categorical([[0.3, 0.7]], 1)

In [15]:
print(a)
print(tf.squeeze(a))

Tensor("categorical/Multinomial:0", shape=(1, 1), dtype=int64)
Tensor("Squeeze:0", shape=(), dtype=int64)


In [0]:
class ProbabilityDistribution(tf.keras.Model):
    def predict(self, logits):
        # sample a random categorical action from given logits
        return tf.random.categorical(logits, 1)

In [0]:
class Model(tf.keras.Model):
    def __init__(self, num_actions):
        super().__init__('mlp_policy')
        
        # no tf.get_variable(), just simple Keras API
        self.hidden1 = kl.Dense(128, activation='relu')
        self.hidden2 = kl.Dense(128, activation='relu')
        self.value = kl.Dense(1, name='value')
        
        # logits are unnormalized log probabilities
        self.logits = kl.Dense(num_actions, name='policy_logits')
        self.dist = ProbabilityDistribution()

    def predict(self, inputs):
        # inputs is a numpy array, convert to Tensor
        x = tf.convert_to_tensor(inputs, dtype=tf.float32)
        
        # separate hidden layers from the same input tensor
        hidden_logs = self.hidden1(x)
        hidden_vals = self.hidden2(x)
        return self.logits(hidden_logs), self.value(hidden_vals)

    def action_value(self, obs):
        # executes call() under the hood
        logits, value = self.predict(obs)
        action = self.dist.predict(logits)
        
        # a simpler option, will become clear later why we don't use it
        # action = tf.random.categorical(logits, 1)
        return np.squeeze(action, axis=-1), np.squeeze(value, axis=-1)

In [19]:
model = Model(num_actions=env.action_space.n)

obs = env.reset()

action, value = model.action_value(obs[None, :])

print(action, value) # [1] [-0.00145713]

Tensor("categorical_2/Multinomial:0", shape=(1, 1), dtype=int64) Tensor("value_1/BiasAdd:0", shape=(1, 1), dtype=float32)


In [0]:
class A2CAgent:
    def __init__(self, model):
        self.model = model

    def test(self, env, render=True):
        obs, done, ep_reward = env.reset(), False, 0
        
        while not done:
            action, _ = self.model.action_value(obs[None, :])
            obs, reward, done, _ = env.step(action)
            ep_reward += reward
            if render:
                env.render()
        return ep_reward

In [21]:
agent = A2CAgent(model)
rewards_sum = agent.test(env, render=True)
print("%d out of 200" % rewards_sum)

AssertionError: ignored